In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
from collections import Counter

In [2]:
# df_train = pd.re
tree_test = ET.parse('data/Laptops_Test_Gold.xml').getroot()
tree_train = ET.parse('data/Laptops_Train_v2.xml').getroot()

In [3]:
def extract_xml(tree):
    text = []
    aspect = []
    for sentence in tree:
        for i, u in enumerate(sentence):
            if i == 0:
                text.append(u.text)
                temp = []
            else:
                for t in u:
                    temp.append(t.attrib)
        aspect.append(temp)
    return text, aspect

In [4]:
text_train, aspect_train = extract_xml(tree_train)

text_test, aspect_test = extract_xml(tree_test)

In [5]:
def get_word_aspect(texts, aspects):
    list_series = []
    for text, aspect in zip(texts, aspects):
        df = pd.DataFrame({"letters": list(text)})
        df["aspect"] = 'O'
        for a in aspect:
            splited = a['term'].split()
            df["aspect"].loc[int(a["from"]):int(a["to"])-1] = 'I'
            df["aspect"].loc[int(a["from"]):int(a["from"])+len(splited[0])] = 'B'
        df["aspect"][df["letters"] == ' '] = ' '
        df = df.sum().str.split()
        df["aspect"] = [x[0] for x in df["aspect"]]
#         print(df)
        list_series.append(df)
#         break
    return list_series

In [6]:
train = get_word_aspect(text_train, aspect_train)
test = get_word_aspect(text_test, aspect_test)

In [7]:
train[0]

letters    [I, charge, it, at, night, and, skip, taking, ...
aspect     [O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, ...
dtype: object

In [8]:
test[0]

letters    [Boot, time, is, super, fast,, around, anywher...
aspect               [B, I, O, O, O, O, O, O, O, O, O, O, O]
dtype: object

In [9]:
word_by_word = pd.DataFrame(train)
len(word_by_word)

3045

In [10]:
# seems that for some case we have not the same amount of word and aspect
# this is certainly due to some error before
# but I will not spend more time on it
# lets just have a fast peak
x = word_by_word.applymap(len)
todrop = (x.letters == x.aspect)
word_by_word = word_by_word[todrop]
word_by_word.head()

,letters,aspect
0,"[I, charge, it, at, night, and, skip, taking, ...","[O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, ..."
1,"[I, bought, a, HP, Pavilion, DV4-1222nr, lapto...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[The, tech, guy, then, said, the, service, cen...","[O, B, I, O, O, O, B, I, O, O, O, O, O, O, O, ..."
3,"[I, investigated, netbooks, and, saw, the, Tos...","[O, O, O, O, O, O, O, O]"
4,"[The, other, day, I, had, a, presentation, to,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [11]:
tot_word = [word for sublist in word_by_word["letters"] for word in sublist]
tot_aspect = [word for sublist in word_by_word["aspect"] for word in sublist]

In [12]:
regrouped = pd.DataFrame({"word": tot_word, "aspect": tot_aspect})
regrouped.head()

,aspect,word
0,O,I
1,O,charge
2,O,it
3,O,at
4,O,night


In [13]:
regrouped.word = regrouped.word.str.replace('[^\w\s]','') # remove punctuations
count = regrouped.groupby(['word'])["aspect"].unique().apply(len).sort_values()[::-1]
print((count == 3).sum(), "words with 3 aspect")
print((count == 2).sum(), "words with 2 aspect")
print((count == 1).sum(), "words with 1 aspect")

97 words with 3 aspect
372 words with 2 aspect
4390 words with 1 aspect


In [14]:
for i, t in regrouped.groupby(['word']):
    if len(t["aspect"].unique()) > 1:
        print(i)
        print("->", t["aspect"].apply(Counter).sum())
# regrouped.groupby(['word'])["aspect"].apply(Counter)

1
-> Counter({'O': 21, 'B': 1})
12
-> Counter({'O': 7, 'B': 1})
13
-> Counter({'O': 9, 'B': 1})
15
-> Counter({'O': 10, 'B': 2})
16
-> Counter({'O': 5, 'I': 1})
17
-> Counter({'O': 5, 'B': 4})
17inch
-> Counter({'O': 3, 'B': 1})
2
-> Counter({'O': 48, 'B': 1})
2008
-> Counter({'O': 2, 'I': 1})
2GB
-> Counter({'B': 2, 'O': 2})
3
-> Counter({'O': 44, 'B': 2, 'I': 1})
30
-> Counter({'O': 4, 'B': 1})
4
-> Counter({'O': 24, 'I': 1})
7
-> Counter({'I': 32, 'O': 9})
800
-> Counter({'O': 2, 'I': 1})
9
-> Counter({'O': 2, 'B': 1})
A
-> Counter({'O': 27, 'I': 1})
AMD
-> Counter({'O': 2, 'B': 1})
ASUS
-> Counter({'O': 6, 'B': 1})
ATI
-> Counter({'O': 1, 'B': 1})
Acer
-> Counter({'O': 19, 'B': 1})
Apple
-> Counter({'O': 64, 'B': 7})
BIOS
-> Counter({'B': 5, 'I': 1})
CD
-> Counter({'O': 2, 'B': 2})
Chrome
-> Counter({'I': 1, 'B': 1})
Core
-> Counter({'B': 1, 'I': 1, 'O': 1})
Customer
-> Counter({'B': 4, 'I': 1})
Cut
-> Counter({'I': 2, 'O': 1})
DELL
-> Counter({'O': 2, 'B': 2})
DVD
-> Counter({'B':

-> Counter({'O': 3, 'I': 1})
handle
-> Counter({'O': 3, 'B': 2})
hard
-> Counter({'B': 38, 'O': 18, 'I': 3})
hardware
-> Counter({'B': 7, 'I': 2})
heat
-> Counter({'O': 5, 'B': 1})
help
-> Counter({'O': 22, 'I': 2})
hinges
-> Counter({'I': 1, 'B': 1})
home
-> Counter({'O': 19, 'I': 1, 'B': 1})
hook
-> Counter({'O': 3, 'B': 2})
i
-> Counter({'O': 93, 'I': 1})
i5
-> Counter({'O': 5, 'B': 1})
i7
-> Counter({'O': 2, 'B': 1})
iMovie
-> Counter({'B': 3, 'O': 1})
iPhoto
-> Counter({'B': 3, 'O': 1})
icon
-> Counter({'O': 1, 'B': 1})
image
-> Counter({'B': 1, 'I': 1, 'O': 1})
in
-> Counter({'O': 327, 'I': 4})
inch
-> Counter({'I': 5, 'O': 5})
included
-> Counter({'O': 5, 'B': 1})
install
-> Counter({'O': 8, 'B': 1})
installation
-> Counter({'O': 1, 'I': 1, 'B': 1})
installed
-> Counter({'O': 10, 'I': 1})
instructions
-> Counter({'O': 2, 'B': 1})
interface
-> Counter({'I': 4, 'B': 1})
internal
-> Counter({'O': 1, 'B': 1})
internet
-> Counter({'B': 15, 'O': 12, 'I': 4})
issues
-> Counter({'O': 17

I am going to stop here for this case

What I would do to go further is clean a bit the word that we extracted so punctuation would not bother.

After I would analyze the count of aspect per words. Maybe we just have to use a simple logsitic regression if the words have always the same aspect.